In [7]:
from sqlalchemy import create_engine
import pandas as pd
import pymysql

In [18]:
# 创建数据库连接
connection = pymysql.connect(
    host='localhost',       # 数据库服务器地址
    user='root',    # 数据库用户名
    password='zy15730850419',# 数据库密码
    database='houseData',   # 要连接的数据库名
    charset='utf8mb4',      # 字符集
    cursorclass=pymysql.cursors.DictCursor  # 使用字典游标
)
with connection.cursor() as cursor:
    # 执行查询
    sql = "SELECT * FROM historyData"
    cursor.execute(sql)

    # 获取查询结果
    result = cursor.fetchall()
    for row in result:
        print(row)

data = pd.read_csv('../data/deal/total_forhistory.csv')
data['街道'].fillna(data['街道'].mode()[0],inplace=True)
# 将DataFrame中的每一行插入到数据库
with connection.cursor() as cursor:
    sql = """
            INSERT INTO historyData (uid, 小区名称, 总价, 单价, 关注度, 所在楼层, 建筑面积, 户型结构, 建筑类型, 房屋朝向, 建筑结构, 装修情况, 配备电梯, 交易权属, 房屋用途, 房屋年限, 产权所属, 抵押信息, 所在城市, 所在街道)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
    values = tuple(row)
    try:
        cursor.execute(sql, values)
    except pymysql.MySQLError as e:
        if e.args[0] == 1205:  # Lock wait timeout exceeded
            print("Lock wait timeout exceeded, retrying...")
            # 重新尝试插入
            cursor.execute(sql, values)
        else:
            raise
    connection.commit()

connection.close()

Lock wait timeout exceeded, retrying...


OperationalError: (1205, 'Lock wait timeout exceeded; try restarting transaction')